In [1]:
import pandas as pd
import numpy as np
import time
import json
import requests
from datetime import date
from matplotlib import pyplot as plt
import datetime
import timeit

In [2]:
# ADD API KEYS HERE - they will all be used in API calls as to avoid rate limiting
api_keys = ['0a9d962918e369aa52bf64266bf11f57',
            'e76fcf746d3ca3cc025c0803dd212fc8',
            '09ce82c5913affec7a666545f702b782',
            '1dcd3965c80843f98a1b195c8ca6b486',
            '3328f91bacb678034ddef11caace433e',
            '2c53f22c76b09beaefd3130d5d1b0105']

In [3]:
county_level_feat = pd.read_csv('county_features_final.csv')

In [4]:
# chosen_county_feat = np.unique(county_level_feat.title.values)
# print(np.unique(county_level_feat['frequency']))
# print(len(chosen_county_feat))
# print(chosen_county_feat)

In [5]:
county_level_feat.head()

,Unnamed: 0,frequency,id,observation_end,observation_start,seasonal_adjustment,title,units,county_id,num_months
0,0,Annual,2020RATIO001001,2018-01-01,2010-01-01,Not Seasonally Adjusted,Income Inequality,Ratio,27336,96
1,1,Monthly,ALAUTA1LFN,2020-02-01,1990-01-01,Not Seasonally Adjusted,Civilian Labor Force,Persons,27336,361
2,2,Annual,ALAUTA1POP,2019-01-01,1970-01-01,Not Seasonally Adjusted,Resident Population,Thousands of Persons,27336,588
3,3,Monthly,ALAUTA1URN,2020-02-01,1990-01-01,Not Seasonally Adjusted,Unemployment Rate,Percent,27336,361
4,4,Annual,ATNHPIUS01001A,2018-01-01,1986-01-01,Not Seasonally Adjusted,All-Transactions House Price Index,Index 2000=100,27336,384


In [6]:
county_level_feat = county_level_feat.sort_values(["county_id", "id"])

counties_list = np.unique(county_level_feat['county_id'].values)
feat_ids = county_level_feat['id'].values

# print(list(counties_list).index(29200))

print(len(counties_list))
# to_retry = np.array([29352,29482,29869])


1985


In [7]:
# edit the start and end variables to determine which counties to get data for
start = 1914
end = 1985
count = start

cur_key_num = 0
num_keys = len(api_keys)
failed_ids = []
for county_id in counties_list[start:end]:
    start_time = timeit.default_timer()
    try:
        time.sleep(.1)
        params = {
            'category_id': county_id,
            'api_key': api_keys[cur_key_num],
            'file_type': 'json'
        }
        cur_key_num = (cur_key_num + 1) % num_keys

        r = requests.get(url = 'https://api.stlouisfed.org/fred/category/series', params=params)
        res = r.json()
        # Get the observations for a series
        seriess = res['seriess']
        res2 = []
        for series in range(len(seriess)):
            current_series = seriess[series]
            series_id = current_series['id']
            freq = current_series['frequency']

            params = {
                'series_id': series_id,
                'api_key': api_keys[cur_key_num],
                'file_type': 'json'
            }
            cur_key_num = (cur_key_num + 1) % num_keys
            time.sleep(.1)

            r = requests.get(url = 'https://api.stlouisfed.org/fred/series/observations', params=params)

            res1 = r.json()
            res1["observations"] = [i for i in res1["observations"] if not (int(i["date"][0:4]) < 2010 or int(i["date"][0:4]) > 2017)]
                
            for val in res1["observations"]:
                del val["realtime_start"]
                del val["realtime_end"]

            if series_id in feat_ids:
                for val in res1['observations']:
                    if (freq == "Annual"):
                        val['feature'] = series_id
                        res2 += [val]
                        for i in range(2, 13):
                            if i < 10:
                                string = '0' + str(i)
                            else:
                                string = str(i)
                            dict_to_add = {}
                            dict_to_add['feature'] = series_id
                            dict_to_add['date'] = val['date'][0:5] + string + val['date'][7:10]
                            dict_to_add['value'] = val['value']
                            res2 += [dict_to_add]
                    elif (freq == "Quarterly"):
                        val['feature'] = series_id
                        res2 += [val]
                        for i in range(1, 3):
                            cur_date = int(val['date'][5:7])
                            date = i + cur_date
                            if date < 10:
                                date = '0' + str(date)
                            else:
                                date = str(date)

                            dict_to_add = {}
                            dict_to_add['feature'] = series_id
                            dict_to_add['date'] = val['date'][0:5] + date + val['date'][7:10]
                            dict_to_add['value'] = val['value']
                            res2 += [dict_to_add]
                    else: #monthly
                        val['feature'] = series_id
                        res2 += [val]

        dates = []
        for i in range(96):
            dates += [res2[i]['date']]

        res3 = {}
        for val in res2:
            if val['feature'] not in res3.keys():
                # new thing
                res3[val['feature']] = [val['value']]
            else:
                res3[val['feature']] += [val['value']]

        for feat in res3:
            if len(res3[feat]) != 96:
                del res3[feat]

        name = 'county_data/' + str(county_id) + '.csv'

        data = pd.DataFrame(res3, index=dates)
        data.to_csv(name)
        elapsed = timeit.default_timer() - start_time
        print('%d. csv written, county id: %s \t time elapsed: %f' %(count, county_id, elapsed))
        count += 1
        
    except:
        print('County ID Failed, rate limited: %s' %(county_id))
        failed_ids += [county_id]
print('Failed counties:')
if (len(failed_ids)) == 0:
    print("None")
else:
    for i in failed_ids:
        print(i)

1914. csv written, county id: 30453 	 time elapsed: 42.607204
1915. csv written, county id: 30454 	 time elapsed: 110.755375
1916. csv written, county id: 30455 	 time elapsed: 46.640505
1917. csv written, county id: 30456 	 time elapsed: 50.932452
1918. csv written, county id: 30457 	 time elapsed: 55.635099
1919. csv written, county id: 30459 	 time elapsed: 53.382909
1920. csv written, county id: 30460 	 time elapsed: 75.379434
1921. csv written, county id: 30461 	 time elapsed: 41.465914
1922. csv written, county id: 30462 	 time elapsed: 69.526356
1923. csv written, county id: 30463 	 time elapsed: 34.087197
1924. csv written, county id: 30464 	 time elapsed: 64.645572
1925. csv written, county id: 30465 	 time elapsed: 36.468822
1926. csv written, county id: 30466 	 time elapsed: 36.657495
1927. csv written, county id: 30467 	 time elapsed: 39.162238
1928. csv written, county id: 30468 	 time elapsed: 64.956128
1929. csv written, county id: 30469 	 time elapsed: 62.272445
1930. c

In [8]:
import os
csvs = os.listdir('county_data')

for idx, elem in enumerate(csvs):
    csvs[idx] = int(elem[:-4])
csvs.sort()
# print(csvs)
# print(list(counties_list))
counties = list(counties_list)
counties.sort()
print(len(csvs))
print(len(counties))
for i in range(len(csvs)):
    if counties[i] != csvs[i]:
        print(counties[i], csvs[i], counties.index(csvs[i]), i)

# print(counties.index(29342))

# print(counties.index(29673))

1985
1985


In [13]:
county_level_feat.head()
title_dict = {}
for index, row in county_level_feat.iterrows():
    title_dict[row["id"]] = row['title']
# print(len(np.unique(county_level_feat['title'])))
# for item in title_dict:
#     print(item, title_dict[item])


73


In [25]:
# code to rename the columns in the csvs
import os
csvs = os.listdir('county_data')

start_time = timeit.default_timer()

# print(csvs)
for idx, file in enumerate(csvs):
    df = pd.read_csv(('county_data/' + file), index_col=0)
    df.rename(columns=title_dict, inplace=True)
    df = df.reindex(sorted(df.columns), axis=1)
    df.to_csv(('county_data/' + file))
    if (idx + 1) % 100 == 0:
        print(idx+1)
elapsed = timeit.default_timer() - start_time
print(elapsed)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
184.18245440000464


In [6]:
# START READING FROM HERE FOR THE ZILLOW DATA STUFF
counties = pd.read_csv('county_table.csv')

# replace county names from the county_table.csv file to match the zillow file county names
# god u have no idea how painful this was
counties['name'] = counties['name'].str.replace('/city', '', regex=False)
counties['name'] = counties['name'].str.replace('/City', '', regex=False)
counties['name'] = counties['name'].str.replace('St.', 'Saint', regex=False)
counties['name'] = counties['name'].str.replace('De Kalb County, GA', 'Dekalb County, GA', regex=False)
counties['name'] = counties['name'].str.replace('De Kalb County, IL', 'DeKalb County, IL', regex=False)
counties['name'] = counties['name'].str.replace('De Kalb County, TN', 'Dekalb County, TN', regex=False)
counties['name'] = counties['name'].str.replace('De Kalb County, MO', 'DeKalb County, MO', regex=False)
counties['name'] = counties['name'].str.replace('Mary\'s', 'Marys', regex=False)
counties['name'] = counties['name'].str.replace('Fond Du Lac County, WI', 'Fond du Lac County, WI', regex=False)
counties['name'] = counties['name'].str.replace('LaFourche', 'Lafourche', regex=False)
counties['name'] = counties['name'].str.replace('Du Page', 'DuPage', regex=False)
counties['name'] = counties['name'].str.replace('Prince George\'s', 'Prince Georges', regex=False)
counties['name'] = counties['name'].str.replace('District of Columbia', 'District of Columbia, DC', regex=False)
counties['name'] = counties['name'].str.replace('Census Area', 'Borough', regex=False)
counties['name'] = counties['name'].str.replace('Borough/municipality', 'Borough', regex=False)
counties['name'] = counties['name'].str.replace('De Soto County, MS', 'DeSoto County, MS', regex=False)
counties['name'] = counties['name'].str.replace('Matanuska-Susitna', 'Matanuska Susitna', regex=False)
counties['name'] = counties['name'].str.replace('Le Flore County, OK', 'LeFlore County, OK', regex=False)
counties['name'] = counties['name'].str.replace('Queen Anne\'s', 'Queen Annes', regex=False)
counties['name'] = counties['name'].str.replace('De Witt County, IL', 'Dewitt County, IL', regex=False)
counties['name'] = counties['name'].str.replace('La Salle Parish', 'LaSalle Parish', regex=False)
counties['name'] = counties['name'].str.replace('O\'Brien County, IA', 'O Brien County, IA', regex=False)
counties['name'] = counties['name'].str.replace('Valdez-Cordova', 'Valdez Cordova', regex=False)


counties.astype({'county_id' : 'int32'})
counties.head()

,county_id,name,state_id
0,27336,"Autauga County, AL",27282
1,27337,"Baldwin County, AL",27282
2,27338,"Barbour County, AL",27282
3,27339,"Bibb County, AL",27282
4,27340,"Blount County, AL",27282


In [7]:
# get the zillow data, and get the county names into the desired format (county_name County, state_abbr) (e.g. 'Fulton County, GA')
zillow = pd.read_csv('County_Zhvi_AllHomes.csv')
zillow.insert(2, 'county_state', zillow['RegionName'] + ', ' + zillow['StateName'])
# add a spot in the zillow dataframe to get the fred county IDs
zillow.insert(0, 'fred_county_id', 0)
# put the fred county IDs in the zillow dataframe
zillow['fred_county_id'] = [counties.loc[counties['name'] == county, 'county_id'].item() for county in zillow['county_state']]
zillow.head()

,fred_county_id,RegionID,SizeRank,county_state,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,...,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
0,27540,3101,0,"Los Angeles County, CA",Los Angeles County,County,CA,CA,Los Angeles-Long Beach-Anaheim,6,...,632055.0,632683.0,634150.0,636295.0,639331.0,641908.0,646084.0,650608.0,657464.0,664495.0
1,27954,139,1,"Cook County, IL",Cook County,County,IL,IL,Chicago-Naperville-Elgin,17,...,247549.0,247860.0,247905.0,247709.0,247416.0,247110.0,246836.0,246704.0,246694.0,246806.0
2,29999,1090,2,"Harris County, TX",Harris County,County,TX,TX,Houston-The Woodlands-Sugar Land,48,...,197084.0,197701.0,198241.0,198605.0,199206.0,199748.0,200350.0,200569.0,201110.0,202143.0
3,27437,2402,3,"Maricopa County, AZ",Maricopa County,County,AZ,AZ,Phoenix-Mesa-Scottsdale,4,...,283650.0,285115.0,286994.0,288669.0,290583.0,292654.0,295076.0,297501.0,299765.0,302433.0
4,27558,2841,4,"San Diego County, CA",San Diego County,County,CA,CA,San Diego-Carlsbad,6,...,601312.0,603078.0,604617.0,606428.0,607862.0,610758.0,614348.0,619380.0,623624.0,628519.0


In [8]:
# drop all columns from the zillow dataset that aren't fred_county_id or zillow price values
zillow.drop(columns=['RegionID', 'SizeRank', 'county_state',
                     'RegionName', 'RegionType', 'StateName', 
                     'State', 'Metro', 'StateCodeFIPS',
                     'MunicipalCodeFIPS'], inplace=True)
columns_list = list(zillow.columns)
columns_list.remove('fred_county_id')

# drop zillow price values outside the date range we care about
zillow.drop(columns=[column for column in columns_list if int(column[0:4]) < 2010 or int(column[0:4]) > 2017],
            inplace=True)

# make the fred_county_ids the indices for the dataframe
fred_county_id = zillow['fred_county_id']
zillow.drop(columns='fred_county_id', inplace=True)
zillow.set_index(fred_county_id, inplace=True)
# transpose the dataframe to get the dates as the index column, just like in the CSVs we got
zillow = zillow.transpose()

zillow.head()

fred_county_id,27540,27954,29999,27437,27558,27551,29215,32078,29955,29232,...,27858,29611,27611,27609,27523,27906,30159,27607,27620,27636
2010-01-31,405646.0,224088.0,136759.0,167180.0,389440.0,518050.0,407994.0,186890.0,132775.0,343972.0,...,55172.0,NaN,46957.0,143286.0,315528.0,101624.0,NaN,NaN,NaN,241237.0
2010-02-28,403964.0,224501.0,136626.0,165897.0,389839.0,517102.0,411915.0,186154.0,132016.0,343042.0,...,54937.0,NaN,46725.0,142846.0,317599.0,100753.0,NaN,NaN,NaN,240367.0
2010-03-31,403863.0,221811.0,136782.0,164421.0,391146.0,516336.0,410430.0,185931.0,131503.0,341512.0,...,54638.0,NaN,46286.0,142614.0,318033.0,99945.0,NaN,NaN,NaN,239781.0
2010-04-30,405270.0,221947.0,137054.0,164001.0,394379.0,520316.0,408956.0,185128.0,131442.0,341091.0,...,54170.0,NaN,46053.0,143124.0,319247.0,99143.0,NaN,NaN,NaN,239066.0
2010-05-31,409305.0,219565.0,137098.0,163301.0,394780.0,522928.0,407316.0,184243.0,131208.0,341285.0,...,53967.0,NaN,45813.0,143948.0,319465.0,98608.0,NaN,NaN,NaN,238346.0


In [9]:
# print(np.unique(zillow[29611.0].values))
# print(zillow[29611.0].isnull().sum())
count2 = 0
drop_cols = []
for col in zillow.columns:
    val = zillow[col].isnull().sum()
    if val != 0:
        print("county: ", end = '')
        print(col, end = '')
        print(" = ", end = '')
        print(val)
        count2 = count2 + 1
        drop_cols += [col]
print('Counties with NaN: ', end = '')
print(count2)
# for col in zillow.columns: 
#     print(col)

county: 27848 = 84
county: 28471 = 43
county: 27888 = 14
county: 29255 = 60
county: 29624 = 31
county: 27837 = 84
county: 29334 = 41
county: 29332 = 60
county: 27431 = 63
county: 28059 = 52
county: 27993 = 1
county: 30124 = 8
county: 30378 = 5
county: 29266 = 60
county: 27377 = 60
county: 28961 = 7
county: 27703 = 60
county: 29450 = 43
county: 29712 = 48
county: 29594 = 88
county: 29152 = 81
county: 27853 = 15
county: 29177 = 60
county: 28462 = 12
county: 28518 = 48
county: 27603 = 42
county: 27576 = 84
county: 28512 = 17
county: 27352 = 50
county: 30261 = 3
county: 546 = 64
county: 29163 = 60
county: 27602 = 60
county: 30386 = 56
county: 988 = 42
county: 29313 = 24
county: 29776 = 57
county: 28671 = 4
county: 29675 = 25
county: 28496 = 7
county: 968 = 48
county: 27355 = 10
county: 576 = 26
county: 30010 = 96
county: 612 = 65
county: 28481 = 10
county: 28263 = 54
county: 30200 = 4
county: 27390 = 44
county: 29815 = 60
county: 28401 = 28
county: 27365 = 7
county: 952 = 54
county: 28813 

In [11]:
# For each csv in the county_data directory, add the zillow price index to the end of the csv.
# Save this with a similar file name in the county_data_zillow directory. If something goes wrong,
# print the name of the failed file.
import os
csvs = os.listdir('county_data')

count2 = 0
count3 = 0
start_time = timeit.default_timer()
for idx, file in enumerate(csvs):
    try:
        df = pd.read_csv(('county_data/' + file), index_col=0)
        df.head()
        df['Zillow Price Index'] = list(zillow[int(file[:-4])])
        df.drop(columns='All-Transactions House Price Index', inplace=True)
        val = df['Zillow Price Index'].isnull().sum()
        if val == 0:
            df.to_csv(('county_data_zillow_2/' + file[:-4] + '_zillow.csv'))
        else:
            count2 += 1
    except:
        print('file {} failed'.format(file))
    if (idx + 1) % 100 == 0:
        print(idx+1)
elapsed = timeit.default_timer() - start_time
print(count2)
print(elapsed)


100
200
300
400
500
file 28277.csv failed
file 28300.csv failed
file 28325.csv failed
600
700
800
file 28909.csv failed
900
file 28996.csv failed
file 29005.csv failed
file 29065.csv failed
1000
file 29166.csv failed
1100
file 29371.csv failed
file 29392.csv failed
1200
1300
file 29723.csv failed
file 29799.csv failed
1400
1500
file 30001.csv failed
file 30133.csv failed
file 30149.csv failed
file 30155.csv failed
1600
1700
1800
file 429.csv failed
file 746.csv failed
1900
file 958.csv failed
148
74.69233229999998


In [27]:
print(count2)
print(count3)

11
0
